In [ ]:
!pip install demucs

In [1]:
!pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

  Cloning https://github.com/facebookresearch/demucs to /tmp/pip-install-p4ihn3qa/demucs_b27a230fc4f24d29abb8044a14d78371
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/demucs /tmp/pip-install-p4ihn3qa/demucs_b27a230fc4f24d29abb8044a14d78371
  Resolved https://github.com/facebookresearch/demucs to commit e976d93ecc3865e5757426930257e200846a520a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.1/87.1 kB 7.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.8/239.8 kB 18.4 MB/

In [2]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import librosa.display
import os
import tarfile

In [3]:

# Path to the dataset tar.gz file
dataset_path = '/content/dev-clean.tar.gz'
# Directory to save extracted files and spectrograms
extracted_dir = '/content/extracted'

spectrogram_dir = '/content/mel_spectrograms'

# Create directories if they don't exist
os.makedirs(extracted_dir, exist_ok=True)
os.makedirs(spectrogram_dir, exist_ok=True)

# Path to the dataset tar.gz file
dataset_path = '/content/dev-clean.tar.gz'
# Directory to extract the files
extracted_dir = '/content/extracted'

# Create the directory if it doesn't exist
os.makedirs(extracted_dir, exist_ok=True)

# Extract the dataset
try:
    with tarfile.open(dataset_path, 'r:gz') as tar:
        tar.extractall(path=extracted_dir)
        print(f"Extraction completed successfully to {extracted_dir}")
except tarfile.ReadError:
    print("Error: Unable to read the tar file. It might be corrupted.")
except EOFError:
    print("Error: The file seems to be incomplete or corrupted.")





Extraction completed successfully to /content/extracted


**LOADING DS AND CREATING SPECTROGRMS **

In [ ]:
import os
import tarfile
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt

# Path to the dataset tar.gz file
dataset_path = '/content/dev-clean.tar.gz'
# Directory to extract the files
extracted_dir = '/content/extracted'
# Directory to save spectrogram images
spectrogram_dir = '/content/mel_spectrograms'

# Create directories if they don't exist
os.makedirs(extracted_dir, exist_ok=True)
os.makedirs(spectrogram_dir, exist_ok=True)

# Step 1: Extract the dataset
try:
    with tarfile.open(dataset_path, 'r:gz') as tar:
        tar.extractall(path=extracted_dir)
        print(f"Extraction completed successfully to {extracted_dir}")
except tarfile.ReadError:
    print("Error: Unable to read the tar file. It might be corrupted.")
except EOFError:
    print("Error: The file seems to be incomplete or corrupted.")

# Step 2: Function to generate and save Mel spectrogram
def save_mel_spectrogram(audio_path, save_path, noise_factor=0.005):
    """
    Generate and save a Mel spectrogram as an image.
    Optionally augment the audio with noise.

    Parameters:
    audio_path (str): Path to the input audio file.
    save_path (str): Path to save the Mel spectrogram image.
    noise_factor (float): Factor controlling the amount of noise for augmentation.
    """
    try:
        # Load the audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))
        augmented_audio = y + noise_factor * noise

        # Generate Mel spectrogram
        S = librosa.feature.melspectrogram(y=augmented_audio, sr=sr, n_mels=128, fmax=8000)
        S_dB = librosa.power_to_db(S, ref=np.max)

        # Save Mel spectrogram as image
        plt.figure(figsize=(10, 4))
        librosa.display.specshow(S_dB, sr=sr, x_axis='time', y_axis='mel')
        plt.colorbar(format='%+2.0f dB')
        plt.title('Mel spectrogram')
        plt.tight_layout()

        # Ensure the directory exists
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path)
        plt.close()
        print(f"Saved Mel spectrogram at {save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Step 3: Process each .flac file in the extracted dataset
def process_dataset(dataset_folder, output_folder, noise_factor=0.005):
    """
    Process each .flac file in a dataset folder, generate Mel spectrograms, and save them as images.

    Parameters:
    dataset_folder (str): Path to the folder containing .flac files.
    output_folder (str): Path to save the spectrogram images.
    noise_factor (float): Factor controlling the amount of noise for augmentation.
    """
    for root, _, files in os.walk(dataset_folder):
        for file in files:
            if file.endswith(".flac"):
                # Input and output paths
                audio_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, dataset_folder)
                spectrogram_save_path = os.path.join(output_folder, relative_path, f"{os.path.splitext(file)[0]}.png")

                # Process and save the spectrogram
                save_mel_spectrogram(audio_path, spectrogram_save_path, noise_factor)

# Example usage
process_dataset(extracted_dir, spectrogram_dir)


Extraction completed successfully to /content/extracted
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0044.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0052.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0000.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0038.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0065.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0003.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0037.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0025.png
Saved Mel spectrogram at /content/mel_spectrograms/LibriSpeech/dev-clean/777/126732/777-126732-0049.png
Saved Me

In [1]:
import librosa
import numpy as np
import soundfile as sf
import os

# Define directories for input and output
extracted_dir = '/content/extracted'  # The directory containing the raw audio files
noisy_audio_dir = '/content/noisy_audio'  # Directory to save noisy audio files
clean_audio_dir = '/content/clean_audio'  # Directory to save clean audio files

# Create directories if they don't exist
os.makedirs(noisy_audio_dir, exist_ok=True)
os.makedirs(clean_audio_dir, exist_ok=True)

# Function to generate and save noisy/clean audio
def save_noisy_and_clean_audio(audio_path, clean_save_path, noisy_save_path, noise_factor=0.005):
    try:
        # Load the clean audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Save clean audio in .flac format
        sf.write(clean_save_path, y, sr)
        print(f"Saved clean audio at {clean_save_path}")

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))  # White noise
        augmented_audio = y + noise_factor * noise  # Add noise to the clean audio

        # Save noisy audio in .flac format
        sf.write(noisy_save_path, augmented_audio, sr)
        print(f"Saved noisy audio at {noisy_save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Loop through all audio files in the extracted directory and process them
found_files = 0  # To track if any files are found
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith(".flac"):  # Process only .flac files
            found_files += 1
            audio_file = os.path.join(root, file)

            # Define paths for saving clean and noisy audio
            clean_audio_path = os.path.join(clean_audio_dir, os.path.basename(file).replace('.flac', '_clean.flac'))
            noisy_audio_path = os.path.join(noisy_audio_dir, os.path.basename(file).replace('.flac', '_noisy.flac'))

            # Generate clean and noisy audio
            save_noisy_and_clean_audio(audio_file, clean_audio_path, noisy_audio_path)

if found_files == 0:
    print("No .flac files found in the extracted directory.")
else:
    print(f"Processed {found_files} .flac files.")


Streaming output truncated to the last 5000 lines.
Saved noisy audio at /content/noisy_audio/7976-105575-0004_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0029_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0029_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0018_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0018_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0021_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0021_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0009_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0009_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0013_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0013_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0020_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0020_noisy.flac
Saved clean audio at /content/clean_aud

In [2]:
import librosa
import numpy as np
import soundfile as sf
import os

# Define directories for input and output
extracted_dir = '/content/extracted'  # The directory containing the raw audio files
noisy_audio_dir = '/content/noisy_audio'  # Directory to save noisy audio files
clean_audio_dir = '/content/clean_audio'  # Directory to save clean audio files

# Target length in samples (5 seconds at 16 kHz)
target_length = 16000 * 5  # 5 seconds of audio at 16kHz sample rate

# Create directories if they don't exist
os.makedirs(noisy_audio_dir, exist_ok=True)
os.makedirs(clean_audio_dir, exist_ok=True)

# Function to generate and save noisy/clean audio with standard size
def save_noisy_and_clean_audio(audio_path, clean_save_path, noisy_save_path, noise_factor=0.005):
    try:
        # Load the clean audio file
        y, sr = librosa.load(audio_path, sr=16000)  # Using 16kHz sample rate

        # Trim or pad the audio to the target length
        if len(y) > target_length:
            y = y[:target_length]  # Trim if longer
        elif len(y) < target_length:
            y = np.pad(y, (0, target_length - len(y)), mode='constant')  # Pad if shorter

        # Save clean audio in .flac format
        sf.write(clean_save_path, y, sr)
        print(f"Saved clean audio at {clean_save_path}")

        # Augment the audio by adding noise
        noise = np.random.randn(len(y))  # White noise
        augmented_audio = y + noise_factor * noise  # Add noise to the clean audio

        # Save noisy audio in .flac format
        sf.write(noisy_save_path, augmented_audio, sr)
        print(f"Saved noisy audio at {noisy_save_path}")
    except Exception as e:
        print(f"Error processing {audio_path}: {e}")

# Loop through all audio files in the extracted directory and process them
found_files = 0  # To track if any files are found
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith(".flac"):  # Process only .flac files
            found_files += 1
            audio_file = os.path.join(root, file)

            # Define paths for saving clean and noisy audio
            clean_audio_path = os.path.join(clean_audio_dir, os.path.basename(file).replace('.flac', '_clean.flac'))
            noisy_audio_path = os.path.join(noisy_audio_dir, os.path.basename(file).replace('.flac', '_noisy.flac'))

            # Generate clean and noisy audio with standard size
            save_noisy_and_clean_audio(audio_file, clean_audio_path, noisy_audio_path)

if found_files == 0:
    print("No .flac files found in the extracted directory.")
else:
    print(f"Processed {found_files} .flac files.")


Streaming output truncated to the last 5000 lines.
Saved noisy audio at /content/noisy_audio/7976-105575-0004_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0029_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0029_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0018_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0018_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0021_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0021_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0009_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0009_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0013_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0013_noisy.flac
Saved clean audio at /content/clean_audio/7976-105575-0020_clean.flac
Saved noisy audio at /content/noisy_audio/7976-105575-0020_noisy.flac
Saved clean audio at /content/clean_aud

In [3]:
!pip install torchaudio soundfile

In [12]:
import torch
import librosa
import soundfile as sf
import os
import numpy as np
from demucs.pretrained import get_model
from demucs.apply import apply_model

class Preprocessing:
    def separate_sources(self, audio_file: str, output_dir: str):
        """
        Use Demucs to perform source separation on a given audio file.
        """
        # Load the pretrained Demucs model
        model = get_model('htdemucs')

        # Move model to GPU if available
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.to(device)

        # Load audio file - now loading as stereo
        wav, sr = librosa.load(audio_file, sr=44100, mono=False)

        print(f"Loaded audio: {audio_file}, shape: {wav.shape}")  # Debug print

        # Convert mono to stereo if necessary
        if wav.ndim == 1:
            wav = np.stack([wav, wav])
        elif wav.shape[0] > 2:
            wav = wav.T
            if wav.shape[0] > 2:  # If still not in the right format
                wav = np.stack([wav[:, 0], wav[:, 0]])

        # Ensure we have exactly 2 channels
        if wav.shape[0] != 2:
            wav = np.stack([wav[0], wav[0]])

        # Convert to tensor
        wav = torch.tensor(wav, device=device)
        wav = wav.unsqueeze(0)  # Add batch dimension

        print(f"Input tensor shape: {wav.shape}")  # Debug print

        # Perform source separation using apply_model
        with torch.no_grad():
            sources = apply_model(model, wav, device=device, overlap=0.25, shifts=2)
            sources = sources.cpu().numpy()

        # Check the shape of separated sources
        print(f"Separated sources shape: {sources.shape}")  # Debug print

        # Save the separated components
        source_names = ['drums', 'bass', 'other', 'vocals']
        for source, name in zip(sources[0], source_names):
            output_path = os.path.join(output_dir, f'{os.path.basename(audio_file)}_{name}.wav')
            print(f"Saving to: {output_path}")  # Debug print
            # Take mean of stereo channels for output
            source_mono = source.mean(axis=0)
            # Ensure the source is properly scaled
            source_mono = source_mono / np.max(np.abs(source_mono)) * 0.9
            sf.write(output_path, source_mono, sr)

        print(f"Separation complete! Files saved in {output_dir}")

    def process_audio(self, input_file: str, output_dir: str):
        """
        Wrapper function to process a single audio file with error handling
        """
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"Processing {input_file}...")

            # Ensure file exists
            if not os.path.exists(input_file):
                print(f"File does not exist: {input_file}")
                return False

            self.separate_sources(input_file, output_dir)
            return True
        except Exception as e:
            print(f"Error processing {input_file}: {str(e)}")
            print(f"Error type: {type(e)}")  # Debug print
            return False

# Example usage
if __name__ == "__main__":
    # Initialize preprocessor
    preprocessor = Preprocessing()

    # Set directory for the noisy audio file and output directory
    audio_file = '/content/noisy_audio/1272-135031-0001_noisy.flac'  # Replace with your audio file path
    output_dir = '/content/clean_audio.flac'  # Replace with your output directory

    # Process the given file
    preprocessor.process_audio(audio_file, output_dir)

    print("Finished processing the audio file.")


Processing /content/noisy_audio/1272-135031-0001_noisy.flac...
Loaded audio: /content/noisy_audio/1272-135031-0001_noisy.flac, shape: (220500,)
Input tensor shape: torch.Size([1, 2, 220500])
Separated sources shape: (1, 4, 2, 220500)
Saving to: /content/clean_audio.flac/1272-135031-0001_noisy.flac_drums.wav
Saving to: /content/clean_audio.flac/1272-135031-0001_noisy.flac_bass.wav
Saving to: /content/clean_audio.flac/1272-135031-0001_noisy.flac_other.wav
Saving to: /content/clean_audio.flac/1272-135031-0001_noisy.flac_vocals.wav
Separation complete! Files saved in /content/clean_audio.flac
Finished processing the audio file.


In [14]:
from IPython.display import Audio

# Path to the denoised audio file
denoised_audio_path = "/content/clean_audio.flac/1272-135031-0001_noisy.flac_vocals.wav"  # Update this path if necessary

# Play the denoised audio file
Audio(denoised_audio_path)

In [15]:
import torch
import librosa
import soundfile as sf
import os
import numpy as np
from demucs.pretrained import get_model
from demucs.apply import apply_model

class Preprocessing:
    def separate_sources(self, audio_file: str, output_dir: str):
        """
        Use Demucs to perform source separation on a given audio file.
        """
        # Load the pretrained Demucs model
        model = get_model('htdemucs')

        # Move model to GPU if available
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        model.to(device)

        # Load audio file - now loading as stereo
        wav, sr = librosa.load(audio_file, sr=44100, mono=False)

        print(f"Loaded audio: {audio_file}, shape: {wav.shape}")  # Debug print

        # Convert mono to stereo if necessary
        if wav.ndim == 1:
            wav = np.stack([wav, wav])
        elif wav.shape[0] > 2:
            wav = wav.T
            if wav.shape[0] > 2:  # If still not in the right format
                wav = np.stack([wav[:, 0], wav[:, 0]])

        # Ensure we have exactly 2 channels
        if wav.shape[0] != 2:
            wav = np.stack([wav[0], wav[0]])

        # Convert to tensor
        wav = torch.tensor(wav, device=device)
        wav = wav.unsqueeze(0)  # Add batch dimension

        print(f"Input tensor shape: {wav.shape}")  # Debug print

        # Perform source separation using apply_model
        with torch.no_grad():
            sources = apply_model(model, wav, device=device, overlap=0.25, shifts=2)
            sources = sources.cpu().numpy()

        # Check the shape of separated sources
        print(f"Separated sources shape: {sources.shape}")  # Debug print

        # Save the separated components
        source_names = ['drums', 'bass', 'other', 'vocals']
        for source, name in zip(sources[0], source_names):
            output_path = os.path.join(output_dir, f'{os.path.basename(audio_file)}_{name}.wav')
            print(f"Saving to: {output_path}")  # Debug print
            # Take mean of stereo channels for output
            source_mono = source.mean(axis=0)
            # Ensure the source is properly scaled
            source_mono = source_mono / np.max(np.abs(source_mono)) * 0.9
            sf.write(output_path, source_mono, sr)

        print(f"Separation complete! Files saved in {output_dir}")

    def process_audio(self, input_file: str, output_dir: str):
        """
        Wrapper function to process a single audio file with error handling
        """
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"Processing {input_file}...")

            # Ensure file exists
            if not os.path.exists(input_file):
                print(f"File does not exist: {input_file}")
                return False

            self.separate_sources(input_file, output_dir)
            return True
        except Exception as e:
            print(f"Error processing {input_file}: {str(e)}")
            print(f"Error type: {type(e)}")  # Debug print
            return False

    def process_dataset(self, noisy_dir: str, output_dir: str):
        """
        Process all audio files in a given directory (noisy audio files).
        """
        noisy_files = sorted([f for f in os.listdir(noisy_dir) if f.endswith('.wav') or f.endswith('.flac')])

        for noisy_file in noisy_files:
            input_file = os.path.join(noisy_dir, noisy_file)
            print(f"Processing file: {input_file}")
            # Call the processing function for each file
            self.process_audio(input_file, output_dir)

# Example usage
if __name__ == "__main__":
    # Initialize preprocessor
    preprocessor = Preprocessing()

    # Set directories for noisy audio files and output directory
    noisy_dir = '/content/noisy_audio'  # Replace with your directory containing noisy files
    output_dir = '/content/output_audio'  # Replace with your output directory

    # Process the entire dataset (all files in noisy_dir)
    preprocessor.process_dataset(noisy_dir, output_dir)

    print("Finished processing all audio files.")


Streaming output truncated to the last 5000 lines.
Processing /content/noisy_audio/6345-93306-0024_noisy.flac...
Loaded audio: /content/noisy_audio/6345-93306-0024_noisy.flac, shape: (220500,)
Input tensor shape: torch.Size([1, 2, 220500])
Separated sources shape: (1, 4, 2, 220500)
Saving to: /content/output_audio/6345-93306-0024_noisy.flac_drums.wav
Saving to: /content/output_audio/6345-93306-0024_noisy.flac_bass.wav
Saving to: /content/output_audio/6345-93306-0024_noisy.flac_other.wav
Saving to: /content/output_audio/6345-93306-0024_noisy.flac_vocals.wav
Separation complete! Files saved in /content/output_audio
Processing file: /content/noisy_audio/6345-93306-0025_noisy.flac
Processing /content/noisy_audio/6345-93306-0025_noisy.flac...
Loaded audio: /content/noisy_audio/6345-93306-0025_noisy.flac, shape: (220500,)
Input tensor shape: torch.Size([1, 2, 220500])
Separated sources shape: (1, 4, 2, 220500)
Saving to: /content/output_audio/6345-93306-0025_noisy.flac_drums.wav
Saving to: /